In [2]:
# Get the Dataset from MS MARCO

In [2]:
from datasets import load_dataset

# Adjust the dataset name/version as necessary
dataset = load_dataset("ms_marco", "v2.1")


In [6]:
# Split it into Training, Validation, and Test Splits

In [3]:
train_testvalid = dataset["train"].train_test_split(test_size=0.2)
test_valid = train_testvalid["test"].train_test_split(test_size=0.5)
dataset_splits = {
    "train": train_testvalid["train"],
    "test": test_valid["test"],
    "valid": test_valid["train"],
}

In [4]:
# Extract Queries and Documents
    #You'll iterate through the data to extract queries and their corresponding relevant documents:

In [6]:
#Generate Triples of Queries, Relevant and Irrelevant Documents
    #This step involves negative sampling. 

In [7]:
import random

def generate_triples(queries, docs, num_negatives=10):
    triples = []
    all_docs = [doc for sublist in docs for doc in sublist]  # Flatten list of docs
    for i, query in enumerate(queries):
        relevant_docs = docs[i]
        for rel_doc in relevant_docs:
            # Sample irrelevant documents
            for _ in range(num_negatives):
                irr_doc = random.choice(all_docs)
                while irr_doc in relevant_docs:
                    irr_doc = random.choice(all_docs)
                triples.append((query, rel_doc, irr_doc))
    return triples

In [8]:
# Tokenize Your Generated Data
# For tokenization, you'll use SentencePiece to generate tokens and then map these tokens using word2vec embeddings.

In [29]:
## HAVING ERROR WHILE SAVING FILE. It seems like because of the dataset structure.
from datasets import DatasetDict

def create_text_file_from_dataset(dataset, filename="ms_marco_texts.txt"):
    with open(filename, "w", encoding="utf-8") as text_file:
        for item in dataset:
            # Write the 'query' to the file if it exists (assuming 'query' is direct key in each item)
            if 'query' in item:
                text_file.write(item['query'] + "\n")
            
            # Now directly iterate over 'passage_text' since it's a list of strings, not a list of dictionaries
            if 'passage_text' in item:
                for passage_text in item['passage_text']:
                    text_file.write(passage_text + "\n")

# Call this function with the subset of data
create_text_file_from_dataset(subset, "ms_marco_texts.txt")

In [33]:
import sentencepiece as spm

spm.SentencePieceTrainer.train('--input=ms_marco_texts.txt --model_prefix=m --vocab_size=100')
sp = spm.SentencePieceProcessor()
sp.load('m.model') 

sentencepiece_trainer.cc(178) LOG(INFO) Running command: --input=ms_marco_texts.txt --model_prefix=m --vocab_size=100
sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: ms_marco_texts.txt
  input_format: 
  model_prefix: m
  model_type: UNIGRAM
  vocab_size: 100
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_file: 
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <un

True

 obj=13.3801 num_tokens=1277 num_tokens/piece=5.25514
unigram_model_trainer.cc(618) LOG(INFO) EM sub_iter=1 size=243 obj=13.0262 num_tokens=1277 num_tokens/piece=5.25514
unigram_model_trainer.cc(618) LOG(INFO) EM sub_iter=0 size=182 obj=14.0565 num_tokens=1462 num_tokens/piece=8.03297
unigram_model_trainer.cc(618) LOG(INFO) EM sub_iter=1 size=182 obj=13.6725 num_tokens=1463 num_tokens/piece=8.03846
unigram_model_trainer.cc(618) LOG(INFO) EM sub_iter=0 size=136 obj=14.6286 num_tokens=1642 num_tokens/piece=12.0735
unigram_model_trainer.cc(618) LOG(INFO) EM sub_iter=1 size=136 obj=14.3438 num_tokens=1642 num_tokens/piece=12.0735
unigram_model_trainer.cc(618) LOG(INFO) EM sub_iter=0 size=110 obj=14.7929 num_tokens=1770 num_tokens/piece=16.0909
unigram_model_trainer.cc(618) LOG(INFO) EM sub_iter=1 size=110 obj=14.5905 num_tokens=1772 num_tokens/piece=16.1091
trainer_interface.cc(687) LOG(INFO) Saving model: m.model
trainer_interface.cc(699) LOG(INFO) Saving vocabs: m.vocab


In [34]:
def tokenize_triples(triples):
    tokenized_triples = []
    for query, rel_doc, irr_doc in triples:
        tokenized_query = sp.encode_as_pieces(query)
        tokenized_rel_doc = sp.encode_as_pieces(rel_doc)
        tokenized_irr_doc = sp.encode_as_pieces(irr_doc)
        tokenized_triples.append((tokenized_query, tokenized_rel_doc, tokenized_irr_doc))
    return tokenized_triples


In [35]:
# For word2vec embeddings, you'll typically load a pre-trained model and map each token to its embedding. If you're using Gensim's word2vec model:

In [42]:
from gensim.models import Word2Vec
import gensim.downloader as api

dataset = api.load("text8")  

# Train a Word2Vec model
model = Word2Vec(sentences=dataset, vector_size=100, window=5, min_count=1, workers=4)

# Save the model to use later
model.wv.save_word2vec_format('word2vec_model.bin', binary=True)

[==================================================] 100.0% 31.6/31.6MB downloaded


In [54]:
from gensim.models import KeyedVectors

word_vectors = KeyedVectors.load_word2vec_format('word2vec_model.bin', binary=True)

def embed_tokens(tokenized_text):
    embeddings = []
    for token in tokenized_text:
        if token in word_vectors:
            embeddings.append(word_vectors[token])
        else:
            embeddings.append(np.zeros(word_vectors.vector_size)) 
    return embeddings


In [37]:
# Implement the Two-Tower architecuture:

In [56]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class TwoTowerModel(nn.Module):
    def __init__(self, pretrained_embedding, embedding_dim, hidden_dim, num_layers):
        super(TwoTowerModel, self).__init__()
        
        # Embedding layer with pre-trained weights.
        self.embedding = nn.Embedding.from_pretrained(pretrained_embedding, freeze=True)  # You can set freeze=False to fine-tune
        
        # RNN encoder for queries.
        self.query_rnn = nn.RNN(embedding_dim, hidden_dim, num_layers, batch_first=True)
        
        # RNN encoder for documents.
        self.doc_rnn = nn.RNN(embedding_dim, hidden_dim, num_layers, batch_first=True)
        
    def forward(self, query_tokens, pos_doc_tokens, neg_doc_tokens):
        # Embed tokens
        query_embeds = self.embedding(query_tokens)
        pos_doc_embeds = self.embedding(pos_doc_tokens)
        neg_doc_embeds = self.embedding(neg_doc_tokens)
        
        # Encode using RNN
        _, query_encoding = self.query_rnn(query_embeds)
        _, pos_doc_encoding = self.doc_rnn(pos_doc_embeds)
        _, neg_doc_encoding = self.doc_rnn(neg_doc_embeds)
        
        # We use only the last hidden state for encoding
        query_encoding = query_encoding[-1]
        pos_doc_encoding = pos_doc_encoding[-1]
        neg_doc_encoding = neg_doc_encoding[-1]
        
        return query_encoding, pos_doc_encoding, neg_doc_encoding

# Assume we have loaded the pretrained word2vec embeddings into a variable named `pretrained_embedding`
# Let's also assume we have the embedding_dim and hidden_dim set, along with num_layers for the RNN

model = TwoTowerModel(pretrained_embedding=pretrained_embedding,
                      embedding_dim=embedding_dim,
                      hidden_dim=hidden_dim,
                      num_layers=num_layers)


NameError: name 'pretrained_embedding' is not defined

In [38]:
# In the model's forward method

In [ ]:
    def forward(self, query_tokens, pos_doc_tokens, neg_doc_tokens):
        # Embed tokens
        query_embeds = self.embedding(query_tokens)
        pos_doc_embeds = self.embedding(pos_doc_tokens)
        neg_doc_embeds = self.embedding(neg_doc_tokens)
        
        # Pass embeddings through the RNN encoder
        _, query_rnn_output = self.query_rnn(query_embeds)
        _, pos_doc_rnn_output = self.doc_rnn(pos_doc_embeds)
        _, neg_doc_rnn_output = self.doc_rnn(neg_doc_embeds)
        
        # Extract the last hidden states as the encodings
        query_encoding = query_rnn_output[-1]
        pos_doc_encoding = pos_doc_rnn_output[-1]
        neg_doc_encoding = neg_doc_rnn_output[-1]
        
        return query_encoding, pos_doc_encoding, neg_doc_encoding

# Initialize the Triplet Loss
criterion = nn.TripletMarginLoss(margin=1.0, p=2)

#  tensors of token indices with shape (batch_size, sequence_length)
query_tokens = torch.tensor([[...], [...], ...])
pos_doc_tokens = torch.tensor([[...], [...], ...])
neg_doc_tokens = torch.tensor([[...], [...], ...])

# Forward pass
query_encoding, pos_doc_encoding, neg_doc_encoding = model(query_tokens, pos_doc_tokens, neg_doc_tokens)

# Compute the loss
loss = criterion(query_encoding, pos_doc_encoding, neg_doc_encoding)

# Backward pass
loss.backward()


In [ ]:
# Loss

import torch
import torch.nn as nn
import torch.optim as optim

class TwoTowerModel(nn.Module):
    def __init__(self, query_encoder, doc_encoder):
        super(TwoTowerModel, self).__init__()
        self.query_encoder = query_encoder
        self.doc_encoder = doc_encoder
    
    def forward(self, query_embeddings, doc_embeddings):
        query_encodings = self.query_encoder(query_embeddings)
        doc_encodings = self.doc_encoder(doc_embeddings)
        return query_encodings, doc_encodings
    
    def compute_distance(self, query_encodings, doc_encodings):
        # Example distance function: Euclidean distance
        distances = torch.norm(query_encodings - doc_encodings, dim=1)
        return distances


In [ ]:
class TripletLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(TripletLoss, self).__init__()
        self.margin = margin
    
    def forward(self, query_encodings, pos_doc_encodings, neg_doc_encodings):
        # Compute distances
        positive_distances = torch.norm(query_encodings - pos_doc_encodings, dim=1)
        negative_distances = torch.norm(query_encodings - neg_doc_encodings, dim=1)
        
        # Compute triplet loss
        losses = torch.relu(positive_distances - negative_distances + self.margin)
        return losses.mean()


In [ ]:
# query_embeddings, pos_doc_embeddings, and neg_doc_embeddings are your data tensors
# query_encoder and doc_encoder are your initialized encoder models

model = TwoTowerModel(query_encoder, doc_encoder)
triplet_loss = TripletLoss(margin=1.0)
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(num_epochs):
    optimizer.zero_grad()
    
    # Forward pass through the model
    query_encodings, pos_doc_encodings = model(query_embeddings, pos_doc_embeddings)
    _, neg_doc_encodings = model(query_embeddings, neg_doc_embeddings)
    
    # Compute loss
    loss = triplet_loss(query_encodings, pos_doc_encodings, neg_doc_encodings)
    
    # Backward pass and optimize
    loss.backward()
    optimizer.step()
    
    print(f"Epoch {epoch+1}, Loss: {loss.item()}")


In [39]:
# Pre-caching Document Encodings

In [ ]:
def cache_doc_encodings(doc_encoder, doc_embeddings):
    with torch.no_grad():  # No need to track gradients
        doc_encodings = doc_encoder(doc_embeddings)
    return doc_encodings

# Assuming `doc_encoder` is your document encoder model and `doc_embeddings` is a tensor of your document embeddings
doc_encodings = cache_doc_encodings(doc_encoder, doc_embeddings)

In [41]:
# Implementing the Inference Function

#For the inference function, we'll encode the incoming query, compute distances between the query encoding and all document encodings, and then select the top k documents with the smallest distances.

In [ ]:
def find_top_k_documents(query_encoder, doc_encodings, query_embedding, k=5):
    with torch.no_grad():  # No need to track gradients during inference
        #Encode the query
        query_encoding = query_encoder(query_embedding.unsqueeze(0))  # Add batch dimension
        
        # Compute distances or similarities
        distances = torch.norm(doc_encodings - query_encoding, dim=1)  # Euclidean distance
        # For cosine similarity, uncomment the following line
        similarities = torch.nn.functional.cosine_similarity(doc_encodings, query_encoding, dim=1)
        
        # Find the top k smallest distances (or largest similarities)
        top_k_indices = torch.topk(distances, k, largest=False).indices  # Change `largest=False` to `True` for similarities
        # For similarities, uncomment the following line
        top_k_indices = torch.topk(similarities, k, largest=True).indices
        
    return top_k_indices.tolist()

# Assuming `query_encoder` is your query encoder model and `query_embedding` is the tensor of your query embedding
top_k_indices = find_top_k_documents(query_encoder, doc_encodings, query_embedding, k=5)


In [ ]:
# Obtain Query Embedding and Retrieve Top 
'''To use 'find_top_k_documents',
To use find_top_k_documents, you need to obtain the embedding for the new query'''

In [ ]:
# Let's assume `tokenizer` and `embeddings` are your preprocessing and embedding tools
# query_text is the new query you received
query_tokens = tokenizer(query_text)  # Tokenize the query text
query_embedding = embeddings(query_tokens)  # Obtain embeddings for the query

# Encode the query and find top k document indices
top_k_indices = find_top_k_documents(query_encoder, doc_encodings, query_embedding, k=5)

# Assuming `documents` is a list of your original documents
top_k_documents = [documents[i] for i in top_k_indices]

print("Top k documents:", top_k_documents)
